In [3]:
import MySQLdb
import MySQLdb.cursors
import requests

class skills_details():
    
    def search_keyword(self,text, keyword_list):
        for word in keyword_list:
            word = str(word)
            if word.title() in text or word.upper() in text or word.capitalize() in text:
                return True
        return False
    
    def load_skill_segment(self,pdf_to_text_list,education_keywords,education_degree_category,
                               project_keywords,skill_keywords, work_experience_keywords,other_keywords,
                              skill_segment):

        # Extract Skill Segment
        for i, text in enumerate(pdf_to_text_list):
            #print(i, text)
            text = text.replace("|","")
            #text = text.encode('ascii', 'ignore')
            flag = False
            if self.search_keyword(text, skill_keywords):
                #print(text)
                skill_segment.append(text)
                i += 1
                flag = True
                while True and i < len(pdf_to_text_list):
                    text = pdf_to_text_list[i]
                    text = text.replace("|","")
                    #print(text)
                    if not self.search_keyword(text, education_keywords) and not self.search_keyword(text,
                                                                                          education_degree_category) and not self.search_keyword(
                            text, project_keywords) and not self.search_keyword(text, work_experience_keywords) and not self.search_keyword(
                            text, other_keywords):
                        skill_segment.append(text)
                    else:
                        break
                    i += 1

            if flag:
                break
        #print("====check skill segment====",skill_segment)
        return skill_segment
                
    def parse_skills(self,sk_seg,sk_data,infoDict):#talent_skills,
        
        #print("check segment", sk_seg)
        
        cnx = MySQLdb.connect(host = 'localhost',
                             database = 'cv_parse',
                             user = 'root',
                             password = 'root', 
                             compress = 1,
                             cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()
        
        skill_segment = sk_seg
        sk = sk_data
        infoDict = infoDict
        infoDict['skill_keyword'] = []
        
        max_score,longest_length = 0,0
        skills_flag = False
        closest_skill_1,closest_skill_2 = "",""
        sl = []
        sk['skills'] = []
        #st = []
        #print(skill_segment)
        #print(len(skill_segment))
        try:  
            if len(skill_segment) > 0 or skill_segment is not None:
                for i in range(0,len(skill_segment)):
                    text = skill_segment[i]
                    #print(text)
                    if not skills_flag:
                        #for skill in programming_skills:
                        text = text.replace("'","")
                        query = """SELECT skills FROM cv_parse.skill_table
                                WHERE '"""+ text + """' LIKE CONCAT('%',skils,'%')"""
                        #print(query)
                        cursor.execute(query)
                        skl = cursor.fetchall()
                        
                        if s:
                            s = skl[0]['skills']
                        #for s in talent_skills:
                            #print(s)
                            #print(skill)

                            #if s in text:
                            sl.append(s)
                            skills_flag = True


                        #for skill in tools:
                        #    if skill in text:
                        #        st.append(skill)
                    if not sl:# or not st:
                        infoDict['skill_keyword'].append(text)
                sk['skills'] = list(set(sl))
        except:
            print("No skills mention")
        print("output skill ===",sk)
        return sk, infoDict


In [4]:
skills_details()

In [ ]:
fsdvs